In [7]:
import pandas as pd
import numpy as np
import time

In [8]:
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')
stop_words = nltk.corpus.stopwords.words("english")
for word in ['what', 'how', 'where', 'who', 'which'] :
    stop_words.append(word)
from string import punctuation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
import spacy

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score
from sklearn.linear_model import LogisticRegression

In [11]:
from sklearn.multioutput import MultiOutputClassifier

In [12]:
file = open("/top_10_tags.txt", "r")
top_10_tags = file.read()
top_10_tags = list(top_10_tags.split('\n')[:-1])
file.close()

In [13]:
data = pd.read_csv("/data.csv")

In [14]:
text = data['Title']
text_spl = text.sample(frac = 0.25).reset_index()
text_spl.head()

,index,Title
0,1139,Expo Cannot Find Module after reinstall
1,4521,How does the following queueMicrotask polyfill...
2,2002,"installation graphviz, no module named graphviz"
3,1733,npm-force-resolutions not working when install...
4,364,Rendering HTML page in CKEditor 5


In [17]:
def preprocess(text) :

    """" Nettoyage du texte :
    passage au minuscule
    suppression du code éventuel du texte que l'on stocke dans une variable 'code'
    suppression et du contenu des balises autres que p (script, alt, ...)
    suppression des balises html
    conservation des textes labellisés par les top 10 tags uniquement
    suppression de la ponctuation, des chiffres,
    et des stopwords
    lemmatisation par spaCy """
    
    text = text.lower()
    
    for i in range(1, len(text)) :
        if text[i-1] == 'c' and text[i] == '#' :
            text = text.replace(text[i], 'sharp')
    
    token_list = nltk.word_tokenize(text)
    
    new_text = []
    
    for token in token_list :
        if token in top_10_tags :
            new_text.append(token)
        elif token not in stop_words :
            for char in token :
                if char in punctuation or char.isdigit() :
                    token = token.replace(char, '')
            new_text.append(token)
    
    lem = nltk.stem.WordNetLemmatizer()
    
    for token in new_text :
        if nltk.pos_tag([token])[0][1].startswith('V') :
            index = new_text.index(token)
            token_lem = lem.lemmatize(token, pos = 'v')
            new_text[index] = new_text[index].replace(token, token_lem)
            
    new_text = ' '.join(new_text)

    return new_text

In [24]:
print("Textes bruts :")
print("")
print(text_spl.loc[:11, 'Title'])
print("---------------------------------------")
print("Textes nettoyés :")
print("")
print(text_spl.loc[:11, 'Title'].apply(preprocess))

Textes bruts :

0               Expo Cannot Find Module after reinstall
1     How does the following queueMicrotask polyfill...
2       installation graphviz, no module named graphviz
3     npm-force-resolutions not working when install...
4                     Rendering HTML page in CKEditor 5
5                      Backup MySQL Database to Dropbox
6     Obviously ambiguous call does not cause a comp...
7          Drawing in JLayeredPane over exising JPanels
8                         PHP string constants overuse?
9               How to update location in for http call
10            Automake Variables to tidy up Makefile.am
11            How do I weak link frameworks on Xcode 4?
Name: Title, dtype: object
---------------------------------------
Textes nettoyés :

0                            expo find module reinstall
1     follow queuemicrotask polyfill fallback use se...
2           installation graphviz  module name graphviz
3           npmforceresolutions work instal new package
4 

In [25]:
%%time
text_clean = text_spl['Title'].apply(preprocess)

CPU times: user 1.64 s, sys: 90.6 ms, total: 1.73 s
Wall time: 2.36 s


In [26]:
text_spl['Title_clean'] = text_clean

In [27]:
data = pd.merge(data.iloc[text_spl['index']], text_spl)[['Title', 'Title_clean', 'Tags']]
data.head(3)

,Title,Title_clean,Tags
0,Expo Cannot Find Module after reinstall,expo find module reinstall,"['java', 'javascript', 'node.js']"
1,How does the following queueMicrotask polyfill...,follow queuemicrotask polyfill fallback use se...,"['java', 'javascript']"
2,"installation graphviz, no module named graphviz",installation graphviz module name graphviz,['python']


In [29]:
#!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 20.5 MB/s eta 0:00:00


In [33]:
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf

In [31]:
# Fonction de préparation des sentences
def bert_inp_fct(sentences, bert_tokenizer, max_length) :
    input_ids=[]
    token_type_ids = []
    attention_mask=[]
    bert_inp_tot = []

    for sent in sentences:
        bert_inp = bert_tokenizer.encode_plus(sent,
                                              add_special_tokens = True,
                                              max_length = max_length,
                                              padding='max_length',
                                              return_attention_mask = True, 
                                              return_token_type_ids=True,
                                              truncation=True,
                                              return_tensors="tf")
    
        input_ids.append(bert_inp['input_ids'][0])
        token_type_ids.append(bert_inp['token_type_ids'][0])
        attention_mask.append(bert_inp['attention_mask'][0])
        bert_inp_tot.append((bert_inp['input_ids'][0], 
                             bert_inp['token_type_ids'][0], 
                             bert_inp['attention_mask'][0]))

    input_ids = np.asarray(input_ids)
    token_type_ids = np.asarray(token_type_ids)
    attention_mask = np.array(attention_mask)
    
    return input_ids, token_type_ids, attention_mask, bert_inp_tot
    

# Fonction de création des features
def feature_BERT_fct(model, model_type, sentences, max_length, b_size, mode='HF') :
    batch_size = b_size
    batch_size_pred = b_size
    bert_tokenizer = AutoTokenizer.from_pretrained(model_type)
    time1 = time.time()

    for step in range(len(sentences)//batch_size) :
        idx = step*batch_size
        input_ids, token_type_ids, attention_mask, bert_inp_tot = bert_inp_fct(sentences[idx:idx+batch_size], 
                                                                      bert_tokenizer, max_length)
        
        outputs = model.predict([input_ids, attention_mask, token_type_ids], batch_size=batch_size_pred)
        last_hidden_states = outputs.last_hidden_state
             
        if step ==0 :
            last_hidden_states_tot = last_hidden_states
            last_hidden_states_tot_0 = last_hidden_states
        else :
            last_hidden_states_tot = np.concatenate((last_hidden_states_tot,last_hidden_states))
    
    features_bert = np.array(last_hidden_states_tot).mean(axis=1)
    
    time2 = np.round(time.time() - time1,0)
    print("temps traitement : ", time2)
     
    return features_bert, last_hidden_states_tot

In [34]:
max_length = 64
batch_size = 10
model_type = 'bert-base-uncased'
model = TFAutoModel.from_pretrained(model_type)
sentences = data['Title_clean'].to_list()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [35]:
features_bert, last_hidden_states_tot = feature_BERT_fct(model, model_type, sentences, 
                                                         max_length, batch_size, mode='HF')

1/1 [==============================] - 3s 3s/step
temps traitement :  357.0


In [40]:
data = data.join(pd.DataFrame(features_bert))

In [62]:
for tag in top_10_tags :
    data['is' + tag] = 0
    index = 0
    for doc_tag in data['Tags'] :
        if not pd.isnull(doc_tag) :
          if tag in doc_tag :
            data.loc[index, 'is' + tag] = 1
        index += 1

In [70]:
X = data.iloc[:,3:-10]
y = data.iloc[:,-10:]

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [72]:
%%time
mclr = MultiOutputClassifier(LogisticRegression(max_iter = 1000)).fit(X_train, y_train)

CPU times: user 2.71 s, sys: 1.93 s, total: 4.64 s
Wall time: 4.69 s


In [73]:
mclr.score(X_train, y_train)

0.6435006435006435

In [74]:
mclr.score(X_test, y_test)

0.31592689295039167

In [76]:
jaccard_score(y_test, mclr.predict(X_test), average = 'micro')

0.24538745387453875